# Privacy on Data Science - AI

## Section: Federated Learning

+ [Lesson: Introducing Federated Learning](#Lesson:-Introducing-Federated-Learning)
+ [Lesson: Basic Remote Execution in PySyft](#Lesson:-Basic-Remote-Execution-in-PySyft)
+ [Project: Learn a Simple Linear Model](#Project:-Learn-a-Simple-Linear-Model)
+ [Lesson: Toy Federated Learning](#Lesson:-Toy-Federated-Learning)

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob") # Pointer to a machine - interface

In [10]:
bob._objects # this a collection of objetcs

{}

In [11]:
x = th.tensor([1,2,3,4,5])

In [12]:
x = x.send(bob) # we are going to send the tensor to bob

In [13]:
bob._objects

{82983404251: tensor([1, 2, 3, 4, 5])}

In [15]:
x # right now this is a pinter to the remote object, and for this case is a kind of tensor 

(Wrapper)>[PointerTensor | me:61157107935 -> bob:82983404251]

In [16]:
x.location

<VirtualWorker id:bob #objects:1>

In [17]:
x.id_at_location

82983404251

In [18]:
x.id

61157107935

In [19]:
x.owner

<VirtualWorker id:me #objects:0>

In [20]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [21]:
x

(Wrapper)>[PointerTensor | me:61157107935 -> bob:82983404251]

In [22]:
x = x.get() # we receive the tensor from bob 
x

tensor([1, 2, 3, 4, 5])

In [23]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [25]:
# try this project here!
alice = sy.VirtualWorker(hook, id="alice")

In [26]:
x = th.tensor([1, 2, 3, 4, 5])

In [27]:
x_ptr = x.send(bob, alice)

In [28]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:90766883333 -> bob:88966817839]
	-> (Wrapper)>[PointerTensor | me:17575046230 -> alice:76003617049]

In [30]:
x_ptr.get() # we have two objects due the relation between the two machines bob and alice

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [31]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob, alice)

In [32]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [27]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [28]:
x

(Wrapper)>[PointerTensor | me:7344279461 -> bob:74976771769]

In [29]:
y

(Wrapper)>[PointerTensor | me:68830170639 -> bob:70981483926]

In [30]:
z = x + y

In [31]:
z

(Wrapper)>[PointerTensor | me:79875157913 -> bob:79875157913]

In [32]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [33]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:28437210120 -> bob:28437210120]

In [34]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [35]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [36]:
z = (x + y).sum()

In [37]:
z.backward()

In [38]:
x = x.get()

In [39]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [40]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [2]:
import torch as th
import syft as sy

In [5]:
hook = sy.TorchHook(th)

In [12]:
bob = sy.VirtualWorker(hook, id="bob")

In [15]:
# try this project here!
input =  th.tensor([[1., 1],[0,1],[1,0],[0,0]], requires_grad = True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad = True).send(bob)

In [19]:
weights = th.tensor([[0.],[0.]], requires_grad = True).send(bob)

In [20]:
bob._objects

{95702988378: tensor([1, 2, 3, 4, 5]), 67931598041: tensor([[1., 1.],
         [0., 1.],
         [1., 0.],
         [0., 0.]], requires_grad=True), 58119980199: tensor([[1.],
         [1.],
         [0.],
         [0.]], requires_grad=True), 56467968431: tensor([[0.],
         [0.]], requires_grad=True)}

In [17]:
input

(Wrapper)>[PointerTensor | me:55256672680 -> bob:67931598041]

In [18]:
target

(Wrapper)>[PointerTensor | me:11952633606 -> bob:58119980199]

In [22]:
for i in range(10):
    
    pred = input.mm(weights)
    
    loss = ((pred - target)**2).sum()
    
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0
    
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [23]:
bob = bob.clear_objects() # clear objects in bob

In [24]:
bob._objects

{}

In [46]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [47]:
bob._objects

{23831414651: tensor([1, 2, 3, 4, 5])}

In [48]:
del x

In [49]:
bob._objects

{}

In [50]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
bob._objects

{92212512027: tensor([1, 2, 3, 4, 5])}

In [52]:
x = "asdf"

In [53]:
bob._objects

{}

In [54]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
x

(Wrapper)>[PointerTensor | me:19384969793 -> bob:49166137090]

In [56]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [57]:
x = "asdf"

In [58]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [59]:
del x

In [60]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [61]:
bob = bob.clear_objects()
bob._objects

{}

In [62]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [63]:
bob._objects

{17426510898: tensor([1, 2, 3, 4, 5])}

In [64]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [65]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

In [130]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [66]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [7]:
from torch import nn, optim
import syft as sy
import torch as th

In [8]:
hook = sy.TorchHook(th)

In [9]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [10]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [11]:
# A Toy Dataset
# Think that this data is our central server
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [49]:
# A Toy Model
model = nn.Linear(2,1)

In [50]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [51]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(3.8478)
tensor(1.0540)
tensor(0.5749)
tensor(0.3653)
tensor(0.2374)
tensor(0.1551)
tensor(0.1017)
tensor(0.0669)
tensor(0.0443)
tensor(0.0294)
tensor(0.0197)
tensor(0.0132)
tensor(0.0090)
tensor(0.0061)
tensor(0.0042)
tensor(0.0029)
tensor(0.0020)
tensor(0.0014)
tensor(0.0010)
tensor(0.0007)


In [12]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [13]:
data_alice = data[0:2].send(alice)
target_alice = target[0:2].send(alice)

In [14]:
data_alice

(Wrapper)>[PointerTensor | me:85498090509 -> alice:86653699342]

In [15]:
target_alice

(Wrapper)>[PointerTensor | me:81274356356 -> alice:30702716304]

In [16]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [17]:
datasets

[((Wrapper)>[PointerTensor | me:72492280833 -> bob:55645041231],
  (Wrapper)>[PointerTensor | me:30268003732 -> bob:37738530640]),
 ((Wrapper)>[PointerTensor | me:85498090509 -> alice:86653699342],
  (Wrapper)>[PointerTensor | me:81274356356 -> alice:30702716304])]

In [68]:
_data, _target =  datasets[0]
_data.location

<VirtualWorker id:bob #objects:0>

In [69]:
_data, _target =  datasets[1]
_data.location

<VirtualWorker id:alice #objects:0>

In [18]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [19]:
train()

tensor(1.2651, requires_grad=True)
tensor(0.0173, requires_grad=True)
tensor(0.0064, requires_grad=True)
tensor(0.0053, requires_grad=True)
tensor(0.0044, requires_grad=True)
tensor(0.0037, requires_grad=True)
tensor(0.0031, requires_grad=True)
tensor(0.0025, requires_grad=True)
tensor(0.0021, requires_grad=True)
tensor(0.0018, requires_grad=True)
tensor(0.0015, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(9.3372e-05, requires_grad=True)
tensor(7.7714e-05, requires_grad=True)
tensor(6.4682e-05, requires_grad=True)
tensor(5

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [20]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [21]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [23]:
x

(Wrapper)>[PointerTensor | me:81403818504 -> bob:76546260607]

In [24]:
x = x.send(alice)

In [25]:
bob._objects

{76546260607: tensor([1, 2, 3, 4, 5])}

In [26]:
alice._objects

{81403818504: (Wrapper)>[PointerTensor | alice:81403818504 -> bob:76546260607]}

In [27]:
x

(Wrapper)>[PointerTensor | me:81333045102 -> alice:81403818504]

In [28]:
y = x + x

In [29]:
y

(Wrapper)>[PointerTensor | me:76396157419 -> alice:15920112218]

In [30]:
bob._objects

{76546260607: tensor([1, 2, 3, 4, 5]),
 82132476458: tensor([ 2,  4,  6,  8, 10])}

In [31]:
alice._objects

{81403818504: (Wrapper)>[PointerTensor | alice:81403818504 -> bob:76546260607],
 15920112218: (Wrapper)>[PointerTensor | alice:15920112218 -> bob:82132476458]}

let's analyze another case the next error 

In [32]:
jon = sy.VirtualWorker(hook, id="jon")

In [34]:
bob.clear_objects()
alice.clear_objects()
jon.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [35]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:alice #objects:1> while the other is on <VirtualWorker id:jon #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

In [36]:
bob.clear_objects()
alice.clear_objects()
jon.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [38]:
bob._objects

{84314758383: tensor([1, 2, 3, 4, 5]), 95501953056: tensor([1, 2, 3, 4, 5])}

In [39]:
alice._objects

{83620059880: (Wrapper)>[PointerTensor | alice:83620059880 -> bob:84314758383]}

In [40]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:83620059880 -> bob:84314758383]

At this moment we get the data from alice 

In [41]:
bob._objects

{84314758383: tensor([1, 2, 3, 4, 5]), 95501953056: tensor([1, 2, 3, 4, 5])}

In [191]:
alice._objects

{}

In [192]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [193]:
bob._objects

{}

Now, we process the data from bob that is connected to alice

In [194]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [195]:
bob._objects

{28843833290: tensor([1, 2, 3, 4, 5])}

In [196]:
alice._objects

{88814770977: (Wrapper)>[PointerTensor | alice:88814770977 -> bob:28843833290]}

In [197]:
del x

In [198]:
bob._objects

{}

In [199]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [1]:
from torch import nn, optim
import syft as sy
import torch as th

In [2]:
hook = sy.TorchHook(th)

In [3]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [4]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [5]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [6]:
bob._objects

{74599015848: tensor([1, 2, 3, 4, 5])}

In [7]:
alice._objects

{59987140254: (Wrapper)>[PointerTensor | alice:59987140254 -> bob:74599015848]}

let's see the application of remote_get method

In [8]:
x.remote_get()

(Wrapper)>[PointerTensor | me:6350692325 -> alice:59987140254]

In [9]:
bob._objects

{}

In [10]:
alice._objects

{59987140254: tensor([1, 2, 3, 4, 5])}

As we can see the last method allows us to get a way to communicate with alice and the object that bob has is now available in alice machine 

Next, the idea is to move the data again to bob

In [11]:
x.move(bob)

(Wrapper)>[PointerTensor | me:6350692325 -> bob:6350692325]

In [12]:
bob._objects

{6350692325: tensor([1, 2, 3, 4, 5])}

In [13]:
alice._objects

{}

let's orchestrate with the data again

In [216]:
x.move(alice)

(Wrapper)>[PointerTensor | me:547575813 -> alice:547575813]

In [217]:
bob._objects

{}

In [218]:
alice._objects

{547575813: tensor([1, 2, 3, 4, 5])}

In [202]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [203]:
bob._objects

{75487377866: tensor([1, 2, 3, 4, 5])}

In [204]:
alice._objects

{94092707138: (Wrapper)>[PointerTensor | alice:94092707138 -> bob:75487377866]}

remote_get

In [205]:
x.remote_get()

(Wrapper)>[PointerTensor | me:87205391815 -> alice:94092707138]

In [206]:
bob._objects

{}

In [207]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}

In [208]:
x.move(bob)

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [209]:
x

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [210]:
bob._objects

{87205391815: tensor([1, 2, 3, 4, 5])}

In [211]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}